In [4]:
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import os
import string
import numpy as np

import joblib

[nltk_data] Downloading package punkt to /home/pkmandke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pkmandke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
%run main.py

Starting clustering...
Done training in 0:00:01.805823s
Time taken 2 days, 17:09:29.201184s


In [ ]:
import joblib
tfidf_obj = joblib.load('../obj/TFIDF_2.sav')

In [ ]:
tfidf_obj.tfidf_matrix.shape

In [ ]:
type(subdocs)

In [ ]:
subdocs[subdocs !=0].shape

In [ ]:
len(tfidf_obj.doc_list)

In [7]:
from agglo_clus import Agglo_clus as ag

In [ ]:
ag_obj = ag(tfidf_obj.tfidf_matrix[:200, :].todense(), doc_names = tfidf_obj.doc_list[:200], num_clus = 10)

In [ ]:
ag_obj.clusterize()
ag_obj.save()

## Testing Agglomerative clustering

In [14]:
dframe = pd.read_json('../data/30Kmetadata.json', orient=str, lines=True)[['identifier-uri',\
                                                                           'contributor-department', 'searchTitle']]
clust = joblib.load('../obj/agglo_clus/iter_1/abstracts_etd_doc2vec_5000_docs_ag_clus.sav')

In [86]:
def setify(depts):
    unq = []
    for st in depts:
        if st in unq:
            continue
        unq.append(st)
    return unq

def get_unique_field(df, field):
    
    return setify(df[field])

def find_docs_in_cluster(df, idx, clust):
    
    return df.loc[df['identifier-uri'].isin([clust.doc_names[_] for _ in idx])]

In [90]:
idx = [idx for idx, _ in enumerate(clust.predictions) if _ == 7]

In [91]:
#clust.doc_names[304]
pd.options.display.max_rows = 5002
depts = [_ for _ in dframe['contributor-department']]

get_unique_field(find_docs_in_cluster(dframe, idx, clust), 'contributor-department')

['Electrical and Computer Engineering ',
 'Mechanical Engineering ',
 'Aerospace and Ocean Engineering ',
 'Mining and Minerals Engineering ',
 'Urban Affairs and Planning ',
 'Power Electronics Systems ',
 'Architecture ',
 'Electrical Engineering ',
 'Materials Science and Engineering ',
 'Biological Systems Engineering ',
 'Civil Engineering ',
 'Computer Science ',
 'Engineering Science and Mechanics ',
 'Industrial and Systems Engineering ',
 'Chemistry ',
 'Chemical Engineering ']

In [5]:
for _ in range(10):
    print(clust.predictions[clust.predictions == _].shape)

(880,)
(481,)
(1001,)
(1015,)
(443,)
(150,)
(314,)
(245,)
(227,)
(211,)


# Trying to generate small clusters

In [6]:
m1 = gensim.models.doc2vec.Doc2Vec.load('../obj/doc2vec/abstracts_etd_doc2vec_5000_docs')

In [9]:
from pre_process import Doc2vec_wrapper, extract_mapped_doc2vecs

doc_vectors, keys = extract_mapped_doc2vecs(m1)

In [10]:

ag_model = ag(doc_vectors, doc_names = keys, num_clus = 100, linkage='ward', affinity='euclidean', iter_='1')

ag_model.clusterize()


Starting clustering...
Done training in 0:00:01.570734s


In [18]:
[len(ag_model.predictions[ag_model.predictions == i]) for i in range(100)]

[76,
 24,
 47,
 38,
 50,
 23,
 19,
 41,
 82,
 30,
 55,
 21,
 10,
 14,
 65,
 31,
 25,
 38,
 15,
 136,
 36,
 50,
 132,
 68,
 40,
 32,
 108,
 101,
 47,
 51,
 59,
 159,
 27,
 96,
 37,
 95,
 61,
 62,
 38,
 46,
 20,
 10,
 115,
 79,
 60,
 51,
 56,
 29,
 57,
 5,
 45,
 60,
 18,
 7,
 123,
 49,
 13,
 47,
 11,
 69,
 114,
 67,
 23,
 14,
 79,
 15,
 82,
 7,
 118,
 35,
 1,
 11,
 54,
 3,
 57,
 343,
 28,
 13,
 64,
 15,
 7,
 45,
 1,
 44,
 40,
 12,
 52,
 59,
 66,
 9,
 49,
 48,
 6,
 65,
 79,
 13,
 71,
 15,
 21,
 13]

# Miscellaneous

In [ ]:
for _ in range(100):
    print(tfidf_obj.tfidf_matrix[_, :][tfidf_obj.tfidf_matrix[_, :] != 0].shape)

In [ ]:
import os
os.getcwd()

In [ ]:
m1 = gensim.models.doc2vec.Doc2Vec.load('../obj/abstracts_etd_doc2vec_5000_docs')

In [ ]:
ranks = []
second_ranks = []
for doc_id in range(1):
    inferred_vector = m1.infer_vector(['a', 'b', 'c', 'd'])
    sims = m1.docvecs.most_similar([inferred_vector], topn=len(m1.docvecs))
    print(len(sims))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [ ]:
m1.docvecs['http://hdl.handle.net/10919/31250 ']